Enter your username (used for marking):

In [1]:
username = 'acp22elc'

# COM4509/6509 Coursework Part 1

Hello,
This is the first of the two parts. Each part accounts for 50\% of the overall coursework mark.

### What to submit

- You need to submit **two jupyter notebooks** (not zipped together), named:

```
assignment_part1_[username].ipynb
assignment_part2_[username].ipynb
```

replacing `[username]` with your username, e.g. `abc18de`.

- Please do not upload the data files used in this Notebook. We just want the two python notebooks.

### Assessment Criteria 

- The marks are indicated for each part: You'll get marks for correct code that does what is asked and gets the right answer. These contribute 45.
- There are also 5 marks for "Code quality" (includes both readability and efficiency).

### Late submissions

We follow the department's guidelines about late submissions, Undergraduate [handbook link](https://sites.google.com/sheffield.ac.uk/comughandbook/your-study/assessment/late-submission). PGT [handbook link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/home/your-study/assessment/late-submission).

### Use of unfair means
"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations." (from the students Handbook).

# A dataset of air quality

We are going to use a dataset of air pollution measurements in Beijing archived by the UCI. To read about the dataset visit the [UCI page](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data).

We are going to:
  1. Preprocess the data
  2. Build our own Lasso-regression
  3. Use sklearn's tools to perform regression on the data
  
Let's get started

In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

We will be trying to predict the pollution (PM2.5) using:
- temperature ('TEMP')
- pressure ('PRES')
- dew-point temperature ('DEWP')
- precipitation ('RAIN')
- wind direction ('wd')
- wind speed ('WSPM')

In [3]:
urllib.request.urlretrieve('https://drive.google.com/uc?id=1m1g4Xn1wMAGV_EU0Nh1HTI1ogA3-tqJk&export=download', './data.csv')

('./data.csv', <http.client.HTTPMessage at 0x14ea99880>)

In [4]:
raw_df = pd.read_csv('data.csv',index_col='No')

#put the columns in a useful order
raw_df = raw_df[['PM2.5', 'year', 'month', 'day', 'hour', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station']]
print(raw_df)

       PM2.5  year  month  day  hour  PM10  SO2   NO2     CO    O3  TEMP  \
No                                                                         
1        9.0  2013      3    1     0   9.0  6.0  17.0  200.0  62.0   0.3   
2       11.0  2013      3    1     1  11.0  7.0  14.0  200.0  66.0  -0.1   
3        8.0  2013      3    1     2   8.0  NaN  16.0  200.0  59.0  -0.6   
4        8.0  2013      3    1     3   8.0  3.0  16.0    NaN   NaN  -0.7   
5        8.0  2013      3    1     4   8.0  3.0   NaN  300.0  36.0  -0.9   
...      ...   ...    ...  ...   ...   ...  ...   ...    ...   ...   ...   
35060   11.0  2017      2   28    19  32.0  3.0  24.0  400.0  72.0  12.5   
35061   13.0  2017      2   28    20  32.0  3.0  41.0  500.0  50.0  11.6   
35062   14.0  2017      2   28    21  28.0  4.0  38.0  500.0  54.0  10.8   
35063   12.0  2017      2   28    22  23.0  4.0  30.0  400.0  59.0  10.5   
35064   13.0  2017      2   28    23  19.0  4.0  38.0  600.0  49.0   8.6   

         PR

Some of the records are missing. We need to handle that before we can easily use the data with most ML tools.

### Question 1: Removing missing data [1 mark]

We are going to handle this by dropping those rows which have a NaN in one of these columns: ['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM'].

Save the result in `nonull_df`.

We can use `df.dropna` to do this. Pandas documentation on this method is [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html).

In [5]:
#Put answer here
nonull_df = raw_df.dropna(axis=0, how='any', subset=['PM2.5', 'hour', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM'])

To check you've done it correctly, you could use `clean_df.isnull().sum()` to confirm that there are no NaN rows in the columns we're interested in:

In [6]:
nonull_df.isnull().sum()

PM2.5        0
year         0
month        0
day          0
hour         0
PM10        17
SO2        242
NO2        331
CO         866
O3         685
TEMP         0
PRES         0
DEWP         0
RAIN         0
wd           0
WSPM         0
station      0
dtype: int64

### Question 2: Removing unwanted columns [1 mark]

Let's remove the columns we're not going to be using. We can use `nonull_df.drop(list_of_column_names, axis=1)` to do this. We will drop: ['year','month','day','PM10','SO2','NO2','CO','O3','station'].

Again, feel free to check if it's worked with `clean_df.isnull().sum()`, for example.

In [7]:
#drop columns year month day PM10 SO2 NO2 CO O3 station
clean_df =  nonull_df.drop(['year', 'month', 'day', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'station'], axis=1)

In [8]:
#there should be 34284 rows left in your dataframe, and 8 columns (note, this was corrected on 7/11/22)
clean_df.shape #=(34284, 8)
print(clean_df)

       PM2.5  hour  TEMP    PRES  DEWP  RAIN   wd  WSPM
No                                                     
1        9.0     0   0.3  1021.9 -19.0   0.0  WNW   2.0
2       11.0     1  -0.1  1022.4 -19.3   0.0  WNW   4.4
3        8.0     2  -0.6  1022.6 -19.7   0.0  WNW   4.7
4        8.0     3  -0.7  1023.5 -20.9   0.0   NW   2.6
5        8.0     4  -0.9  1024.1 -21.7   0.0  WNW   2.5
...      ...   ...   ...     ...   ...   ...  ...   ...
35060   11.0    19  12.5  1013.5 -16.2   0.0   NW   2.4
35061   13.0    20  11.6  1013.6 -15.1   0.0  WNW   0.9
35062   14.0    21  10.8  1014.2 -13.3   0.0   NW   1.1
35063   12.0    22  10.5  1014.4 -12.9   0.0  NNW   1.2
35064   13.0    23   8.6  1014.1 -15.9   0.0  NNE   1.3

[34284 rows x 8 columns]


### Question 3: Splitting the dataset [2 marks]

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. **It is important to remember that the test data has to be set aside before preprocessing**.

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage. Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Later we will be performing a grid search to select parameter values. To do this we'll do cross-validation, but rather than split the data into training and validation here we'll split it later. So for now we'll just split into:

- The training (and validation) set will have 85% of the total observations, 
- The test set, the remaining 15%.

To avoid unwanted correlations connecting the training and test, we will split these by time. So:

- Take the first 85% of the rows from clean_df and put them in train_df, take the remaining 15% of the rows and put them in test_df

In [9]:
#Put answer here
#split first 85% into train, last 15% into test
split_size = int(len(clean_df)*0.85)
train_df = clean_df[:split_size]
test_df = clean_df[split_size:]


To check the sizes are correct, we can use:

In [10]:
len(train_df)/len(clean_df),len(test_df)/len(clean_df)

(0.8499883327499709, 0.15001166725002918)

# Detour: Lasso Regression

Later we will use the sklearn toolkit, but in this section **you will develop your own code** to do the Lasso regression.

### Ordinary Least Squares Regression

First, let's just perform normal linear regression.

We'll use a toy design matrix & labels to use to check our code works. We'll also specify a weight vector too, for testing.

In [11]:
X = np.array([[0.0,0],[1,3],[2.2,3]])
y = np.array([0.0,1,2])
w = np.array([1.0,2])

### Question 4: Prediction Function [2 marks]

The first task is to write a function to make predictions. Can you complete this function for linear regression, i.e. the predictions for all our points $f(X,\mathbf{w}) = X \mathbf{w}$. [corrected: 7/11/22, don't need $X$ tranposed]

In [12]:
def prediction(X,w):
    #Put answer here
    return np.dot(X,w)

In [13]:
#You can use this code to check you've written the right function
np.all(prediction(X,w)==np.array([0. , 7. , 8.2])) #Should return 'True'

True

### Question 5: Objective Function [4 marks]

Now we need to write a function that returns the 'error'.
We'll just do normal Ordinary Least Squares with linear regression, so if you remember the cost function for that is:

$$E = \sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2$$

Where $E$ is the error, $N$ the number of points, $y_i$ is one of the labels, $\mathbf{x}_i$ is the input for that label, $\mathbf{w}$ is the weight vector. $f$ is the prediction function you've already written. Or feel free to substitute in $\mathbf{x}_i^\top \mathbf{w}$.

In [14]:
def objective(X,y,w):
    """
    Computes the sum squared error (for us to perform OLS linear regression).
    """
    #Put answer here
    #return np.sum((y-(np.dot(X,w)))**2)
    f = X@w
    resid = y - f
    return resid.T @ resid



In [15]:
#You can use this code to check you've written the right function
print (objective(X,y,w))
objective(X,y,w)==74.44

74.44


True

### Question 6: Objective Function Gradient [4 marks]

Now you need to find the derivative of the objective wrt the parameter vector. You've already done this in lectures, so remember the gradient of the error function (for linear regression) is:

$$\frac{\partial E}{\partial \mathbf{w}} = 2 X^\top X \mathbf{w} - 2 X^\top y$$

Add code to do this here:

In [16]:
def objective_derivative(X,y,w):
    """
    Computes the derivative of the sum squared error, wrt the parameters.
    """
    #Put answer here   
    return 2*X.T@(X@w) - 2*X.T@y
objective_derivative(X,y,w)

array([39.28, 73.2 ])

To check your gradients are correct, we can estimate the gradient numerically:

In [17]:
def numerical_objective_derivative(X,y,w):
    """
    Computes a numerical approximation to the derivative of the sum squared error, wrt the parameters.
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective(X,y,w+d)-objective(X,y,w-d))/2e-6
    return g

The two gradient vectors should be approximately equal:

In [18]:
objective_derivative(X,y,w),numerical_objective_derivative(X,y,w)

(array([39.28, 73.2 ]), array([39.28, 73.2 ]))

### Question 7: Optimise $\mathbf{w}$ to minimise the objective [4 marks]

Now you need to use the gradient function you've written to maximise $\mathbf{w}$ using gradient descent.
Start with a sensible choice of w. You'll need to loop lots of times (e.g. 1000). At each iteration: compute the gradient and subtract the scaled gradient from the w parameter (you'll need to scale it by the learning rate, of e.g. 0.01).

In [19]:
def optimise_parameters(X,y,startw):
    """
    Returns the w that minimises the objective.
    """
    #Put answer here
    w = startw
    for i in range(5000):
        w = w - objective_derivative(X,y,w)*0.01
    return w

In [20]:
bestw = optimise_parameters(X,y,w)
print(bestw) #print our solution

[0.83333333 0.05555556]


Let's compare this to the answer provided by sklearn:

In [21]:
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=False)
clf.fit(X,y)
print(clf.coef_) #matches the value of w we found above, hopefully!

[0.83333333 0.05555556]


## Lasso Regression

### Question 8: New Objective Function [3 marks]

We're now going to regularise the regression using $L_1$ regularisation - i.e. Lasso regression.

We need a **new objective function**:

$$E = \frac{1}{2N}\sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2 + \alpha \sum_{j=1}^P |w_j|$$

This is similar to before (but the first term is now half the mean squared error, rather than the sum squared error). The second term is the L1 regularisation term.

In [22]:
def objective_lasso(X,y,w,alpha):
    """
    Computes half the mean squared error, with an additional L1 regularising term. alpha controls the level of regularisation.
    """
    
    #Put answer here
    hmse = np.sum((y-(np.dot(X,w)))**2)/(len(y)*2)
    l1 = alpha * np.sum(np.abs(w))
    return hmse + l1

### Question 9: The gradient of the lasso regression objective [3 marks]

The tricky bit the derivative of the objective.

The first part is similar to before. So, with the regularising term, the derivative is:
$$\frac{\partial E}{\partial \mathbf{w}} = \frac{1}{N}(X^\top X \mathbf{w} - X^\top y) + \alpha\;\text{sign}(\mathbf{w})$$

where $\text{sign}(\mathbf{w})$ returns a vector of the same shape as $\mathbf{w}$ with +1 if the element is positive and -1 if it's negative. The `np.sign` method does this for you.

Have a think about why this is (think about what differentiating the 'absolute' function $|w_j|$ involves - think about what happens when it's positive vs when it's negative. 

In [23]:
def objective_lasso_derivative(X,y,w,alpha):
    """
    Returns the derivative of the Lasso objective function.
    """
    #Put answer here
    return (X.T@(X@w) - X.T@y)/len(y) + (np.sign(w)*alpha)

We can check it again, numerically. The two pairs of parameters should be the same:

In [24]:
def numerical_objective_lasso_derivative(X,y,w,alpha):
    """
    This finds a numerical approximation to the true gradient
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective_lasso(X,y,w+d,alpha)-objective_lasso(X,y,w-d,alpha))/2e-6
    return g

objective_lasso_derivative(X,y,w,0.1),numerical_objective_lasso_derivative(X,y,w,0.1)

(array([ 6.64666667, 12.3       ]), array([ 6.64666667, 12.3       ]))

### Question 10: Optimise $\mathbf{w}$ to minimise the Lasso objective [2 marks]

As before we need to optimise to find the optimum value of $\mathbf{w}$, for this Lasso objective.
You'll need to loop lots of times (e.g. 5000). Start with a sensible choice of w. At each iteration: compute the gradient and subtract the scaled gradient from the w parameter (you'll need to scale it by the learning rate, of e.g. 0.05).

In [25]:
def optimise_parameters_lasso(X,y,startw):
    """
    Returns the w that minimises the Lasso objective.
    """
    #Put answer here
    w = startw
    alpha = 0.1
    for i in range(5000):
        w = w - objective_lasso_derivative(X,y,w,alpha)*0.05
    return w

optimise_parameters_lasso(X,y,w)

array([0.63888889, 0.14259259])

We can check against the sklearn method:

In [26]:
clf = linear_model.Lasso(alpha=0.1,fit_intercept=False)
clf.fit(X,y)
print(clf.coef_)

[0.63931263 0.1423666 ]


The above result should approximately match the one you computed.

# Back to air pollution

### Question 11: One-hot-encoding [4 marks]

One of the columns isn't numerical, but instead is a string type: The wind direction. The best way to deal with this is one-hot-encoding.

pandas has a tool for doing this: `pd.get_dummies(series, prefix='prefix_to_use')`. In our example the series is: `clean_df.wd`.

You'll need to:
1. Make the one-hot encoding table using the code above.
2. Delete the `wd` column from our table (hint: you did this earlier for other columns).
3. Join the one hot data to the table. To do this use something like `dataframe1.join(dataframe2)`.



In [27]:
def add_wd_onehot(df):
    """Add new one-hot encoding set of columns, removes the old column it's made from. Returns new dataframe."""
    #Put answer here
    # Get one hot encoding of columns 'vehicleType'
    dataframe2 = pd.get_dummies(df['wd'])
    
    # Drop column as it is now encoded
    df = df.drop('wd',axis = 1)

    # Join the encoded df
    df = df.join(dataframe2)
    return df

#you could use this code to see if it's worked?
train_df_wdencoded = add_wd_onehot(train_df)
train_df_wdencoded

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,WSPM,E,ENE,ESE,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
No,,,,,,,,,,,,,,,,,,,,,
1,9.0,0,0.3,1021.9,-19.0,0.0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,11.0,1,-0.1,1022.4,-19.3,0.0,4.4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,8.0,2,-0.6,1022.6,-19.7,0.0,4.7,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,8.0,3,-0.7,1023.5,-20.9,0.0,2.6,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,8.0,4,-0.9,1024.1,-21.7,0.0,2.5,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29760,138.0,23,28.2,997.0,24.2,0.0,1.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29761,145.0,0,27.7,997.1,24.3,0.0,0.9,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29762,139.0,1,26.9,996.6,24.6,0.0,0.8,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Question 12: Standardise the data [3 marks]
[note: updated from term 'Normalise' to 'Standardise' on 7/11/22. For clarity, I want the mean to be zero and the standard deviation to be one].

Now we need to standardise [edit: corrected] the data.

You could manipulate just some columns by using, for example: `df.iloc[:,1:]` - this returns a dataframe that consists of all but the first column.

Feel free to use either tools from `sklearn.preprocessing` or standardise [edit: corrected on 7/11/22] it by, for example, using the mean and the standard deviation of the columns by calling `some_dataframe.mean()` or `some_dataframe.std()`.

In [28]:
from sklearn.preprocessing import StandardScaler
def standardise(df):
    """
    Returns a new dataframe in which all but the PM2.5 columns are standardised (i.e. have a mean of zero and standard deviation of 1)
    [note: the function name used to be 'normalise' but was modified for clarity]

    [addition: 7/11/22
    Think about if you want to standardise using the *training* data's mean and standard deviation (for the test data).
    """
    #Put answer here
    #manually standardise
    #temp = df.iloc[:,0]
    #df = df.drop(df.columns[0],axis=1)
    #df = (df - df.mean())/df.std()
    #df.insert(0, 'PM2.5', temp)
    
    #using sklearn
    df1 = df.iloc[:,0]
    df2 = df.iloc[:,1:]
    scaler = StandardScaler()
    df2 = pd.DataFrame(scaler.fit_transform(df2), columns = df.columns[1:], index = df.index)
    df2.insert(0, 'PM2.5', df1)
    df = df2
    return df

train_df_preprocessed = standardise(add_wd_onehot(train_df))
test_df_preprocessed = standardise(add_wd_onehot(test_df))
#print(add_wd_onehot(train_df))
print(train_df_preprocessed)


       PM2.5      hour      TEMP      PRES      DEWP      RAIN      WSPM  \
No                                                                         
1        9.0 -1.657991 -1.244548  1.080110 -1.612190 -0.084708  0.194043   
2       11.0 -1.513672 -1.280171  1.128008 -1.634231 -0.084708  2.157165   
3        8.0 -1.369353 -1.324700  1.147168 -1.663619 -0.084708  2.402555   
4        8.0 -1.225034 -1.333606  1.233384 -1.751782 -0.084708  0.684824   
5        8.0 -1.080716 -1.351418  1.290862 -1.810558 -0.084708  0.603027   
...      ...       ...       ...       ...       ...       ...       ...   
29760  138.0  1.661344  1.240186 -1.305218  1.561690 -0.084708 -0.623924   
29761  145.0 -1.657991  1.195657 -1.295639  1.569037 -0.084708 -0.705721   
29762  139.0 -1.513672  1.124410 -1.343537  1.591078 -0.084708 -0.787518   
29763  128.0 -1.369353  1.097692 -1.372276  1.605772 -0.084708 -1.114705   
29764  116.0 -1.225034  1.079881 -1.381855  1.591078 -0.084708 -0.705721   

           

Here we put the training and test inputs (X) and outputs (y) into four variables:

In [29]:
X = train_df_preprocessed.iloc[:,1:].to_numpy()
y = train_df_preprocessed.iloc[:,0].to_numpy()

Xtest = test_df_preprocessed.iloc[:,1:].to_numpy()
ytest = test_df_preprocessed.iloc[:,0].to_numpy()

We can use the same code we wrote before, using the Lasso from sklearn to fit the data. Here we'll turn fit_intercept on, as we've not added a '1's column to our design matrix.

So feel free to use:
```
clf = linear_model.Lasso(alpha=0.1,fit_intercept=True)
clf.fit(X,y)
```

### Question 13: Finding the RMSE of the Lasso regressor predictions [2 marks]

Next compute the **RMSE** of the predictions for (a) the training data and (b) the test data.
The RMSE (root mean squared error) could be computed, for example with:

```np.sqrt(np.mean((predicted_values-true_values)**2))```

In [30]:
#Put answer here
clf = linear_model.Lasso(alpha=0.1,fit_intercept=True)
clf.fit(X,y)
#a
predicted_values = clf.predict(X)
true_values = y
rmse_lasso_train = np.sqrt(np.mean((predicted_values - true_values)**2))
#b
predicted_values_test = clf.predict(Xtest)
true_values_test = ytest
rmse_lasso_test = np.sqrt(np.mean((predicted_values_test - true_values_test)**2))
print(rmse_lasso_train,rmse_lasso_test)

72.40451930761152 88.63540499735817


We can compare this to the standard deviation of the data, we should do better than that!

In [31]:
np.std(y), np.std(ytest)

(82.43136001277581, 103.25109103178704)

### Question 14: Random Forest [8 marks]

The final step is to use a random forest regressor.

If we use the default random forest regressor, we find we get considerable over-fitting. So we need to explore different parameters. We will use a cross-validated grid search over the parameters:

- max_features: The number of features to consider when looking for the best split (i.e. controls subsampling), *from 1 to the number of features* in 4 steps (e.g. use `np.linspace`)
- n_estimators: The number of trees in the forest, from 10 to 100 in 4 steps.
- max_samples : the number of samples to draw from to train each base estimator, from 0.1 to 0.9 in 4 steps.

We will use `GridSearchCV`.

Have a look at the documentation for this, the three parameters we need to specify are:

- the 'estimator': an **INSTANCE** of RandomForestRegressor.
- param_grid: a **DICTIONARY**, each item is the title of the parameter, and equals an array of the values we need to test. For exmaple, one of the items might be `{'max_samples': np.linspace(0.1, 0.9, 5)}`.
- You'll need to think carefully how to make the lists for the `max_features` and `n_estimators` as these both need to be (positive) integers. E.g. use `.astype(int)`.

In [32]:
#Put answer here:
#1. Create a grid of parameter values for n_estimators, max_features and max_samples,
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
n_estimators_opts = np.linspace(10, 100, 4, dtype=int)
max_features_opts = np.linspace(1, X.shape[1], 4, dtype=int)
max_samples_opts = np.linspace(0.1, 0.9, 4)
param_grid = {'n_estimators': n_estimators_opts, 'max_features': max_features_opts, 'max_samples': max_samples_opts}
#2. Create a GridSearchCV object, using the random forest regressor:
rf = RandomForestRegressor()
grid_regression = GridSearchCV(rf, param_grid, scoring='neg_root_mean_squared_error')

#Note: Because there is so much training data, using the full dataset takes too long. So here we'll just use 10%
np.random.seed(42)
idx = np.sort(np.random.choice(len(X), size=int(len(X)*0.1), replace=False))
#3. Fit to training data in (the subset of) X and y
grid_regression.fit(X[idx,:],y[idx])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 1,  8, 15, 22]),
                         'max_samples': array([0.1       , 0.36666667, 0.63333333, 0.9       ]),
                         'n_estimators': array([ 10,  40,  70, 100])},
             scoring='neg_root_mean_squared_error')

Here we print the best parameters from the grid search (on the training/validation cross-validation run):

In [33]:
best_n_estimators = grid_regression.best_params_['n_estimators']
best_max_features = grid_regression.best_params_['max_features']
best_max_samples = grid_regression.best_params_['max_samples']
print('Best n_estimator', best_n_estimators)
print('Best max features', best_max_features)
print('Best max samples', best_max_samples)

Best n_estimator 40
Best max features 8
Best max samples 0.6333333333333333


### Question 15: RMSE for the Random Forest Regressor [1 mark]

Finally compute the RMSE for the training and test data:

In [34]:
#Put answer here
rf = RandomForestRegressor(n_estimators=best_n_estimators, max_samples=best_max_samples, max_features=best_max_features,random_state=42)
rf.fit(X,y)
#a
predicted_values = rf.predict(X)
true_values = y
rmse_rf_train = np.sqrt(np.mean((predicted_values - true_values)**2))
#b
predicted_values_test = rf.predict(Xtest)
true_values_test = ytest
rmse_rf_test = np.sqrt(np.mean((predicted_values_test - true_values_test)**2))
print(rmse_rf_train,rmse_rf_test)



31.33178385455363 80.29388321949091


We can compare this to the standard deviations for the two sets of data.

In [35]:
np.std(y), np.std(ytest)

(82.43136001277581, 103.25109103178704)

### Question 16: Did the random forest do better than lasso regression? [1 mark]



In [36]:
#Put answer here
#Random forest did better than lasso regression for the training and tetest data